# Python DSA & Data Science Assessment

This notebook contains clear, well-structured solutions to the given problems, written in a straightforward, student-style manner with explanations.

## Question 1: Sliding Window Anomaly Score

We compute the anomaly score as the absolute difference between the current value and the median of the previous `k` values. To do this efficiently, we use two heaps:
- A max heap for the lower half
- A min heap for the upper half

This allows median calculation in `O(log k)` time per step.

In [1]:

import heapq
from collections import defaultdict

def sliding_window_anomaly(arr, k):
    max_heap = []  # lower half (as negative values)
    min_heap = []  # upper half
    delayed = defaultdict(int)

    def balance():
        # balance heap sizes
        if len(max_heap) > len(min_heap) + 1:
            heapq.heappush(min_heap, -heapq.heappop(max_heap))
        elif len(min_heap) > len(max_heap):
            heapq.heappush(max_heap, -heapq.heappop(min_heap))

    def get_median():
        if k % 2 == 1:
            return -max_heap[0]
        else:
            return (-max_heap[0] + min_heap[0]) / 2

    # initialize first window
    for x in arr[:k]:
        heapq.heappush(max_heap, -x)
        heapq.heappush(min_heap, -heapq.heappop(max_heap))
        balance()

    result = []

    for i in range(k, len(arr)):
        median = get_median()
        result.append(abs(arr[i] - median))

        # remove outgoing element
        out = arr[i - k]
        if out <= -max_heap[0]:
            delayed[out] += 1
        else:
            delayed[out] += 1

        # add new element
        x = arr[i]
        if x <= -max_heap[0]:
            heapq.heappush(max_heap, -x)
        else:
            heapq.heappush(min_heap, x)

        balance()

    return result

# Example
arr = [10, 20, 30, 40, 100]
k = 3
print(sliding_window_anomaly(arr, k))


[20, 80]


## Question 2: Top-K Correlated Feature Pairs

We calculate the Pearson correlation for every pair of features (columns). To improve efficiency:
- Means and standard deviations are precomputed
- A min-heap of size `k` keeps track of the top correlations

In [2]:

import math
import heapq

def top_k_correlated_features(data, k):
    n = len(data)
    m = len(data[0])

    # transpose data to work with columns
    cols = list(zip(*data))

    means = [sum(col) / n for col in cols]
    stds = [
        math.sqrt(sum((x - means[i]) ** 2 for x in cols[i]) / n)
        for i in range(m)
    ]

    heap = []

    for i in range(m):
        for j in range(i + 1, m):
            cov = sum(
                (cols[i][r] - means[i]) * (cols[j][r] - means[j])
                for r in range(n)
            ) / n

            if stds[i] == 0 or stds[j] == 0:
                continue

            corr = cov / (stds[i] * stds[j])
            val = abs(corr)

            if len(heap) < k:
                heapq.heappush(heap, (val, i, j, corr))
            elif val > heap[0][0]:
                heapq.heappushpop(heap, (val, i, j, corr))

    return [(i, j, round(c, 3)) for _, i, j, c in sorted(heap, reverse=True)]

# Example
data = [[1,2,3],[2,4,6],[3,6,9],[4,8,12]]
k = 1
print(top_k_correlated_features(data, k))


[(0, 2, 1.0)]
